In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%cd "../.."

/home/hazzu/Code/opencv_streamlit


In [37]:
from services.keypoint_matching.superpoint import SuperPointFrontend

fe = SuperPointFrontend(
    weights_path="./services/keypoint_matching/superpoint_v1.pth",
    nms_dist=4,
    conf_thresh=0.015,
    nn_thresh=0.7,
    cuda=True,
)

Run on GPU


/home/hazzu/Code/opencv_streamlit/services/keypoint_matching/superpoint.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(weights_path

In [59]:
import os
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
from services.semantic_keypoint_detection.services import DATATYPES

bf = cv2.BFMatcher()
accuracy_rotations_superpoint = []

cache_image = []
cache_descriptors = []
for i in range(8):
    cache_image.append([])
    cache_descriptors.append([])
    for j in tqdm(range(500), f"Load cache {i}"):
        image = cv2.imread(os.path.join(DATATYPES[i], "images", f"{j}.png"))
        cache_image[i].append(image)

        original_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        original_gray = original_gray.astype(np.float32) / 255.0

        _, original_descriptors, __ = fe.run(original_gray)
        if original_descriptors is not None:
            original_descriptors = original_descriptors.T

        cache_descriptors[i].append(original_descriptors)


for alpha in range(0, 360, 10):
    accuracy_superpoint = []
    for i in range(8):
        for j in tqdm(range(500), f"Run {alpha} {i}"):
            # Read image
            image = cache_image[i][j]

            # Rotate image
            rotate_matrix = cv2.getRotationMatrix2D(
                ((image.shape[1] - 1) / 2.0, (image.shape[0] - 1) / 2.0), alpha, 1.0
            )
            image_rotated = cv2.warpAffine(
                image, rotate_matrix, (image.shape[1], image.shape[0])
            )

            # Convert to grayscale
            rotated_gray = cv2.cvtColor(image_rotated, cv2.COLOR_BGR2GRAY)
            rotated_gray = rotated_gray.astype(np.float32) / 255.0

            # Extract keypoints and descriptors
            rotated_keypoints, rotated_descriptors, __ = fe.run(rotated_gray)

            # Match descriptors
            original_descriptors = cache_descriptors[i][j]
            if original_descriptors is None or rotated_descriptors is None:
                accuracy_superpoint.append(0)
            else:
                rotated_descriptors = rotated_descriptors.T
                matches = bf.knnMatch(original_descriptors, rotated_descriptors, k=2)

                good_matches = 0
                for k in range(len(matches)):
                    if len(matches[k]) == 2:
                        if matches[k][0].distance < 0.75 * matches[k][1].distance:
                            good_matches += 1

                accuracy_superpoint.append(
                    good_matches
                    / max(len(original_descriptors), len(rotated_descriptors))
                )

    accuracy_rotations_superpoint.append(np.mean(accuracy_superpoint))
    print(np.mean(accuracy_superpoint))

Load cache 0:   0%|          | 0/500 [00:00<?, ?it/s]

/home/hazzu/anaconda3/envs/opencv_streamlit/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Run 350 7: 100%|██████████| 500/500 [00:01<00:00, 309.92it/s]

[np.float64(0.95675), np.float64(0.6379210261433168), np.float64(0.5351469338596009), np.float64(0.3526185700977042), np.float64(0.17203088424864466), np.float64(0.08575832560626423), np.float64(0.08065691972227722), np.float64(0.09260794024868914), np.float64(0.10182209994746456), np.float64(0.08831165057651069), np.float64(0.11346384896000884), np.float64(0.11612084614585944), np.float64(0.11590449198401107), np.float64(0.11092617189660715), np.float64(0.11096156344197053), np.float64(0.12341631170142903), np.float64(0.15162514984058173), np.float64(0.16825043589123168), np.float64(0.16504264741467423), np.float64(0.16810613361873633), np.float64(0.1499020610158607), np.float64(0.12343988502626053), np.float64(0.11309517258031983), np.float64(0.11442928853907094), np.float64(0.11370604459881248), np.float64(0.11562272477412694), np.float64(0.11065479837897942), np.float64(0.08818149172441067), np.float64(0.10460775207333262), np.float64(0.09669381630740483), np.float64(0.081298023983

In [60]:
print(accuracy_rotations_superpoint)

[np.float64(0.95675), np.float64(0.6379210261433168), np.float64(0.5351469338596009), np.float64(0.3526185700977042), np.float64(0.17203088424864466), np.float64(0.08575832560626423), np.float64(0.08065691972227722), np.float64(0.09260794024868914), np.float64(0.10182209994746456), np.float64(0.08831165057651069), np.float64(0.11346384896000884), np.float64(0.11612084614585944), np.float64(0.11590449198401107), np.float64(0.11092617189660715), np.float64(0.11096156344197053), np.float64(0.12341631170142903), np.float64(0.15162514984058173), np.float64(0.16825043589123168), np.float64(0.16504264741467423), np.float64(0.16810613361873633), np.float64(0.1499020610158607), np.float64(0.12343988502626053), np.float64(0.11309517258031983), np.float64(0.11442928853907094), np.float64(0.11370604459881248), np.float64(0.11562272477412694), np.float64(0.11065479837897942), np.float64(0.08818149172441067), np.float64(0.10460775207333262), np.float64(0.09669381630740483), np.float64(0.081298023983

In [61]:
np.save(
    "./services/keypoint_matching/accuracy_rotations_superpoint.npy",
    accuracy_rotations_superpoint,
)